<a href="https://colab.research.google.com/github/yoossi/Cocktail-recipe-by-ingredient/blob/main/Cocktails_by_Ingredients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting cocktail recipes by ingredients

I love making and drinking cocktails, but there are too many cocktail choices available, and I sometimes forget which cocktails I can make with ingredients I have. I decided to do a simple project to solve this problem.

Every recipe is in different format. Some require only 2 ingredients, while other may require 5 ingredients. Some have garnish, and some don't. Therefore, I decided to use document oriented database, because it would be much easier to transform the receipe to data than relational database. 

TinyDB is document oriented and file based database, which is the perfect choice for this project

In [1]:
#install tinyDB
!pip install tinydb

In [2]:
## import libraries
from tinydb import TinyDB, Query, where
import tinydb
import json
import pprint
import random

I already have created my cocktail receipes as json, and saved it on my github.

In [3]:
!wget https://raw.githubusercontent.com/yoossi/cocktail/main/cocktails.json

--2022-03-05 02:03:28--  https://raw.githubusercontent.com/yoossi/cocktail/main/cocktails.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4789 (4.7K) [text/plain]
Saving to: ‘cocktails.json’

cocktails.json      100%[===================>]   4.68K  --.-KB/s    in 0s      

2022-03-05 02:03:29 (56.0 MB/s) - ‘cocktails.json’ saved [4789/4789]



The json file is list of dictionaries, so I had to iterate the list and insert each dictionary into the tinydb database

In [4]:
# create a new storage for the database
db = tinydb.TinyDB("db_storage.json")  

# open the json file, and iterate over each entry and insert into database
with open("./cocktails.json", "r") as f:  
    json_data = json.load(f)  

for entry in json_data:  
    db.insert(entry)  


In [5]:
# display number of recipes
print('Number of cocktail recipes: ',len(db))

Number of cocktail recipes:  15


This is what a cocktail recipe looks like.

In [6]:
# display one of the randomly chosen recipe
pprint.pprint(db.all()[random.randint(0,len(db)-1)])

{'ingredient': [{'amount': '2 oz', 'name': 'Whiskey', 'type': 'Spirit'},
                {'amount': '0.75 oz', 'name': 'Simple Syrup', 'type': 'Syrup'},
                {'amount': '0.75 oz', 'name': 'Lemon Juice', 'type': 'Juice'}],
 'method': 'Shake',
 'name': 'Whiskey Sour'}


It will later ask user for input as integer corresponding to the ingredient name, to avoid any misspelling. 
To acomplish this task, I got all ingredient names from all recipes, and converted into dictionary with integer label. This way, it will generate the new dictionary even when a new ingredient is added.

In [7]:
# Get all recipe documents which has ingredient key 
results = db.search(Query().ingredient.exists())
ingredient = set()
# iterate recipe documents and add to list
for result in results:
  for ing in result['ingredient']:
    ingredient.add(ing['name'])

ingredient = sorted(ingredient)
# convert to dictionary with label as key and ingredients as value
ingredients = {i+1: key for i, key in enumerate(ingredient)}

Using the ingredient names, it asks for user input for list of ingredients.

In [8]:
active = True
inputs=[]
# show the 
print("Ingredient list")
for k,v in ingredients.items():
  print(k,":", v)
# get user input until user stops
while active:
  userinput = input("Which Ingredient do you have? (type # or type N to stop): ")
  if userinput == 'N':
    print("Input Completed")
    active = False
  else:
    # input validation
    try:
      val = int(userinput)
      if val >= 0 and val <= len(ingredients):
        inputs.append(ingredients[val])
      else:
        print("Input is out of range")
    except ValueError:
      print("Please enter number")

Ingredient list
1 : Bitters
2 : Coffee Liqueur
3 : Cola
4 : Espresso Shot
5 : Gin
6 : Ginger Beer
7 : Lemon Juice
8 : Lime Juice
9 : Milk
10 : Orange Liqueur
11 : Rum
12 : Simple Syrup
13 : Tequila
14 : Tonic Water
15 : Vodka
16 : Whiskey
Which Ingredient do you have? (type # or type N to stop): 13
Which Ingredient do you have? (type # or type N to stop): 16
Which Ingredient do you have? (type # or type N to stop): 11
Which Ingredient do you have? (type # or type N to stop): 3
Which Ingredient do you have? (type # or type N to stop): 1
Which Ingredient do you have? (type # or type N to stop): 10
Which Ingredient do you have? (type # or type N to stop): 12
Which Ingredient do you have? (type # or type N to stop): N
Input Completed


Finally, with given list of ingredients, it searches the database for possible cocktail recipes.

In [9]:
results = db.search(Query().ingredient.all(Query().name.one_of(inputs)))
print('The avaiable cocktails with current ingredients are')
# if there is none
if len(results) == 0:
  print('None')
else:
  for result in results:
    print("-", result['name'])

The avaiable cocktails with current ingredients are
- Cuba Libre
- Old Fashioned
